In [1]:
import tensorflow as tf

# https://github.com/patrick-kidger/tools
import tools

from base import ml

In [2]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [3]:
### DNN hyperparameters
hidden_units = [100] * 7
# i.e. the number of fine grid points:
logits = (ml.fine_grid_fineness.t + 1) * (ml.fine_grid_fineness.x + 1)
processor = ml.ScaleCenter()
activation = tf.nn.crelu

### Training hyperparameters #1
drop_rate = 0.0
drop_type = 'dropout'
gradient_clip = 1.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir_base = './saved_models/grid_multi_peakon/'
uuid = None
model_dir = ml.model_dir_str(model_dir_base, hidden_units, logits,
                             processor=processor, activation=activation, 
                             uuid=uuid)

dnn_factory = ml.DNNFactory(hidden_units, logits, drop_rate=drop_rate,
                            drop_type=drop_type, model_dir=model_dir, 
                            log_steps=log_steps, activation=activation, 
                            gradient_clip=gradient_clip, batch_norm=False,
                            processor=processor)

max_steps = 0
if TRAIN:
    ### Training hyperparameters #2
    batch_size = 50
    max_steps += 5000
    with ml.BatchData.context(gen_one_data=ml.gen_peakons_on_grid, 
                              batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory()
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)

    batch_size = 500
    max_steps += 5000
    with ml.BatchData.context(gen_one_data=ml.gen_peakons_on_grid, 
                              batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory()
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)

    batch_size = 1500
    max_steps += 3000
    with ml.BatchData.context(gen_one_data=ml.gen_peakons_on_grid, 
                              batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory()
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)

AttributeError: type object 'BatchData' has no attribute 'from_func'